This is a script for simulating arbitrary activity and export its results.

In [62]:
%matplotlib notebook
import IPython.display
from utils_demo import *
from sys import platform
import sys
from PIL import Image
import matplotlib.pyplot as plt
import json
import rdflib
import glob
import os
import re
import copy
import time
import random
from timeout_decorator import timeout, TimeoutError

sys.path.append('../simulation')
sys.path.append('../dataset_utils/')

import numpy as np
import random
import cv2
import add_preconds
import evolving_graph.check_programs as check_programs
import evolving_graph.utils as utils

from unity_simulator.comm_unity import UnityCommunication

In [63]:
comm = UnityCommunication(timeout_wait=45)

Select scene number. 
Please see https://github.com/xavierpuigf/virtualhome/tree/master/simulation#environment .

If you want to use the existing activity data provided by VirtualHome, you can load the dataset.
Please see the below link for getting the dataset. (Note that there are many errors in the dataset.)
https://github.com/xavierpuigf/virtualhome#dataset

In [64]:
scene = 1
scene_graph = "TrimmedTestScene" + str(scene) + "_graph"
executable_program_path = "../../VirtualHome2KG/dataset/programs_processed_precond_nograb_morepreconds/executable_programs/" + scene_graph + "/*/*.txt"
executable_program_list = []
for file_path in glob.glob(executable_program_path):
    executable_program_list.append(file_path.replace("../../VirtualHome2KG/dataset/programs_processed_precond_nograb_morepreconds/executable_programs/" + scene_graph + "/", ""))

The HomeOntology is used for randomly selecting an activity.
Load HomeOntology as RDF graph.
If you do not select an activity randomly, you do not have to use the HomeOntology.

In [65]:
rdf_g = rdflib.Graph()
rdf_g.parse("../ontology/vh2kg_ontology.ttl", format="ttl")

<Graph identifier=N38a5ec3f939c47e7a5c7561796e3bc96 (<class 'rdflib.graph.Graph'>)>

In [66]:
def get_activity_from_ontology(activity_type):
    results = []
    qres = rdf_g.query(
    """
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX : <http://www.owl-ontologies.com/VirtualHome.owl#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
select ?activity where { 
    ?activity rdfs:subClassOf :%s .
 } 
       """ % activity_type)

    result = ""
    for row in qres:
        activity = "".join(row).replace("http://www.owl-ontologies.com/VirtualHome.owl#","")
        arr = activity.split("_")
        arr[0] = arr[0].capitalize()
        activity = " ".join(arr)
        result = activity
        results.append(result)
    return results

In [67]:
def generate_list_of_steps(file_path):
    file = open(file_path, "r", encoding="utf-8")
    i = 0
    list_of_steps = []
    program_name = ""
    description = ""
    char= "<char0>"
    while True:
        line = file.readline()
        if line:
            line = line.replace("\n","")
            if i==0:
                program_name = line
            elif i==1:
                description = line
            elif line.startswith("["):
                list_of_steps.append(line)
            else:
                pass
            i+=1
        else:
            break
    return program_name, description, list_of_steps

In [68]:
def get_activity_program(category):
    #unexecutable = ["Take shower", "Take shoes off", "Wash teeth", "Wash face", "Dust", "Clean toilet", "Clean room", "Scrubbing living room tile floor is once week activity for me", "Clean mirror", "Play games", "Play on laptop", "Read on sofa"]
    unexecutable = []
    executable = []
    activities = get_activity_from_ontology(category)
    for activity_name in activities:
        if activity_name in unexecutable:
            continue
        results = [program for program in program_list if program["name"] == activity_name]
        if len(results) == 0:
            print("Nothing: " + activity_name)
        else:
            print("Success: " + activity_name)
            executable.append({"activity_name": activity_name, "results": results})
    return executable

In [69]:
data_path = "../../VirtualHome2KG/dataset/programs_processed_precond_nograb_morepreconds/withoutconds/*/*.txt"
program_list = []
for file_path in glob.glob(data_path):
    file_name = file_path.replace("../../VirtualHome2KG/dataset/programs_processed_precond_nograb_morepreconds/withoutconds/", "")
    if file_name in executable_program_list:
        program_name, description, list_of_steps = generate_list_of_steps(file_path)
        program_list.append({
            "file_name":file_name,
            "name": program_name,
            "description": description,
            "list_of_steps": list_of_steps
        })

In [70]:
executable_activity_list = []
for activity_class in ["BedTimeSleep", "EatingDrinking", "FoodPreparation", "GettingReady", "HouseArrangement", "HouseCleaning", "HygieneStyling", "Leisure", "Other", "PhysicalActivity", "SocialInteraction", "Work"]:
    executable_activity_list.extend(get_activity_program(activity_class))

Nothing: Bed time
Nothing: Put alarm clock in bedroom
Nothing: Put them to sleep
Nothing: Read them bedtime story
Success: Take nap
Success: Go to sleep
Nothing: Wake me up
Success: Sleep
Nothing: Go to bed
Nothing: Get ready for bed
Success: Eat
Nothing: Eat on fancy china
Nothing: Eat breaksfast
Nothing: Eat dessert
Nothing: Eat ice cream
Success: Eat dinner
Nothing: Eat cheese
Success: Drink
Nothing: Get something to drink
Nothing: Have dinner
Nothing: Have ice coffee
Nothing: Dance for table as we eat
Nothing: Eat cereal
Nothing: Eat while watching tv
Nothing: Eat donuts
Nothing: Dining
Nothing: Eat snacks and drink tea
Nothing: Eat family meals
Success: Have snack
Nothing: Eat meals
Nothing: Fix bowl of cereal
Nothing: Fix sandwhich
Nothing: Make peanut butter sandwhich
Success: Make coffee
Nothing: Cut steak
Nothing: Make sandwitch
Nothing: Defrost chicken
Nothing: Make eggs
Nothing: Fix sandwich
Nothing: Make sandwich
Success: Breakfast
Nothing: Make ice in ice trays for freezer

In [71]:
unsupport_unity_exec_time = {
    "PutOn": 10.0,
    "PutOff": 10.0,
    "Drop": 2.0,
    "PlugIn": 5.0,
    "PlugOut": 5.0,
    "Sleep": 21600.0,
    "Wake": 5.0
}

In [72]:
def check_unsupport_action(script):
    flag = True
    for line in script:
        m = re.search(r'\[.+\]', line)
        action = m.group().replace('[', '')
        action = action.replace(']', '')
        if action in [x for x in unsupport_unity_exec_time.keys()]:
            flag = False
            break
    return flag

In [73]:
def switchOff(g, class_names):
    for class_name in class_names:
        objects = find_nodes(g, class_name=class_name)
        for obj in objects:
            states = [x for x in obj["states"] if x != "ON"]
            if "OFF" not in states:
                states.append("OFF")
            obj["states"] = states
#             for node in g["nodes"]:
#                 if node["id"] == obj["id"]:
#                     node = obj
        return g

In [74]:
def export(activity_name, graph_state_list, activity_cnt, time_list, script, object_name_map, out_dir_name):
    state_cnt = 0
    for graph_state in graph_state_list:
        state_cnt += 1
        file_path = "graph_state_list_with_bbox_png/" + out_dir_name + "_scene" + str(scene) + "/"  + activity_name + "/" + activity_cnt + "/activityList-graph-state-" + '{0:03d}'.format(state_cnt) + ".json"
        with open(file_path, 'w') as outfile:
            json.dump(graph_state, outfile)

    with open("graph_state_list_with_bbox_png/" + out_dir_name + "_scene" + str(scene) + "/" + activity_name + "/" + activity_cnt + "/activityList-program.txt", 'w') as f:
        for s in script:
            #ここでscriptのobject_nameを置換する処理書く
            for x in object_name_map:
                if x in s:
                    s = s.replace("<" + x  + ">", "<" + object_name_map[x] + ">")
            f.write("%s\n" % s)

    with open("graph_state_list_with_bbox_png/" + out_dir_name + "_scene" + str(scene) + "/" + activity_name + "/" + activity_cnt  + "/program-description.txt", 'w') as f:
        f.write("%s\n" % activity["name"])
        f.write("%s\n" % activity["description"])
        
    time_list = [str(time) for time in time_list]
    duration = "\n".join(time_list)
    with open("graph_state_list_with_bbox_png/" + out_dir_name + "_scene" + str(scene) + "/" + activity_name + "/" + activity_cnt + "/duration.txt", 'w') as f:
        f.write(duration)

In [75]:
def update_bbox(pre_graph, current_graph):
    try:
        new_graph= copy.deepcopy(current_graph)
        
        for pre_node in pre_graph["nodes"]:
            for new_node in new_graph["nodes"]:
                if new_node["id"] == pre_node["id"]:
                    new_node["bounding_box"] = pre_node["bounding_box"]
           
    except Exception as e:
        print(e)
    
    return new_graph

In [76]:
def update_states(pre_graph, graph_state, object_id_map):
    try:
        new_graph= copy.deepcopy(pre_graph)
        
        skip = [object_id_map[x] for x in object_id_map]
        for new_node in new_graph["nodes"]:
            new_node_id = new_node["id"]
            if new_node_id in skip:
                continue
            temp_node_id = copy.deepcopy(new_node_id)
            # scriptで使用したobject_idとgraph_stateで反映されたidが異なる場合
            if str(new_node_id) in object_id_map:
                temp_node_id = object_id_map[str(new_node_id)]
            for graph_state_node in graph_state["nodes"]:
                if temp_node_id == graph_state_node["id"]:
                    new_node["states"] = graph_state_node["states"]
                    new_node["properties"] = graph_state_node["states"]
           
    except Exception as e:
        print(e)
    
    return new_graph

In [77]:
def create_object_id_map(original_script, executed_program, g):
    try:
        object_id_map = {}
        object_name_map = {}
        for original_block_str, executed_block_str in zip(original_script, executed_program):
            original_action, original_obj_names_corr, original_inst_nums_corr = augmentation_utils.parseStrBlock(original_block_str)
            executed_action, executed_obj_names_corr, executed_inst_nums_corr = augmentation_utils.parseStrBlock(executed_block_str)
            
            #object_idのmapを作る
            for original_id, executed_id in zip(original_inst_nums_corr, executed_inst_nums_corr):
                if original_id not in object_id_map:
                    if original_id != executed_id:
                        object_id_map[original_id] = int(executed_id)
            
            #object_nameのmap作る
            for original_obj_name, original_id in zip(original_obj_names_corr, original_inst_nums_corr):
                g_objects = [x for x in g["nodes"] if x["id"] == original_id]
                if len(g_objects) != 0:
                    g_obj = g_objects[0]
                    if original_obj_name != g_obj["class_name"]:
                        object_name_map[original_obj_name] = g_obj["class_name"]
    except Exception as e:
        print(e)
    return object_id_map, object_name_map

In [78]:
def delete2000(graph_state_list):
    new_graph_state_list = []
    for graph_state in graph_state_list:
        new_nodes = [x for x in graph_state["nodes"] if x["id"] < 2000]
        new_edges = [x for x in graph_state["edges"] if x["from_id"] < 2000 and x["to_id"] < 2000]
        new_graph_state_list.append({"nodes": new_nodes, "edges": new_edges})
    return new_graph_state_list

In [79]:
@timeout(40)
def time_managed_render_script(instruction, find_solution, image_width, recording, image_height, file_name_prefix, frame_rate, vis_check_object, vis_check_character, out_graph):
    success, message = comm.render_script(script=[instruction],
#                                               processing_time_limit=60,
                                              find_solution=find_solution,
                                              image_width=image_width,
                                              image_height=image_height,  
                                              #skip_animation=False,
                                              recording=True,
#                                               gen_vid=True,
                                              #save_scene_states=True,
                                              file_name_prefix=file_name_prefix,
                                              output_folder="Output/",
                                              frame_rate=frame_rate,
                                              vis_check_object=vis_check_object, 
                                              vis_check_character=vis_check_character, 
                                              out_graph=out_graph
                                             )
    return success, message

In [81]:
out_dir_name = "test20220715"
rand_executable_activity_list = random.sample(executable_activity_list, len(executable_activity_list))
for executable_activity in rand_executable_activity_list:
    activity_list = executable_activity["results"]
    activity_name = executable_activity["activity_name"]
    activity_cnt = 0
    try:
        # if activity_name != "Relax on sofa": 
        #     continue
        os.mkdir("graph_state_list_with_bbox_png/" + out_dir_name + "_scene" + str(scene) + "/" + activity_name)
        for activity in activity_list:
            if activity_cnt > 0:
                break
            try:
                new_graph_state_list = []
                time_list = []
                comm.reset(scene-1)
                comm.add_character('chars/Female2')
                success, g = comm.environment_graph();
                print(activity["file_name"])
                script = activity["list_of_steps"]
                print(script)
                executable_program = []
                for x in script:
                    executable_program.append("<char0> " + x)
                executable_program

                # check if supported action
                if check_unsupport_action(script) == False:
                    continue
                    
#                 #sit
#                 sit = False
#                 for s in script:
#                     if "Sit" in s:
#                         sit = True
#                         continue
#                 if sit:
#                     continue
                
                preconds = add_preconds.get_preconds_script(script).printCondsJSON()
                
                print(preconds)
                print("switch off")
                g = switchOff(g, ["computer", "tv", "faucet", "television"])
                
                g = check_programs.translate_graph_dict_nofile(g)
                info = check_programs.check_script(script, preconds, graph_path=None, inp_graph_dict=g)
                message, final_state, graph_state_list, graph_dict, id_mapping, info, helper, executed_script = info
                # if delete randomly placced objects 
                graph_state_list = delete2000(graph_state_list)
                print(message)
                executed_program = []
                for x in executed_script:
                    executed_program.append("<char0> " + re.sub("\[\d+\]","",x.__str__()).strip())

                if message != "Script is executable":
                    continue
                
                new_script = []
                for x in script:
                    new_script.append("<char0> " + x)
                    
                i = 0
                
                object_id_map, object_name_map = create_object_id_map(new_script, executed_program, g)
                print("object_id_map")
                print(object_id_map)
                
                success, g = comm.environment_graph()
                g = update_states(g, graph_state_list[i], object_id_map)
                new_graph_state_list.append(copy.deepcopy(g))

                for instruction in executed_program:
                    i += 1
                    start = time.time()
#                     try:
#                         success, g = time_managed_render_script(instruction,False,480,360,True,activity_name + str(activity_cnt), 15,False,False,False)
#                     except TimeoutError as e:
#                         print(e)
#                         break;
#                     finally:
#                         print("掛った時間", time.time() - start)
                    success, message = comm.render_script(script=[instruction],
#                                               processing_time_limit=60,
                                              find_solution=False,
                                              image_width=480,
                                              image_height=360,  
                                              #skip_animation=False,
                                              recording=True,
#                                               gen_vid=True,
                                              #save_scene_states=True,
                                              file_name_prefix=activity_name + str(activity_cnt),
                                              output_folder="Output/",
                                              frame_rate=15,
                                              vis_check_object=False, 
                                              vis_check_character=False, 
                                              out_graph=False
                                             )
                    time_list.append(time.time() - start)
                    success, g = comm.environment_graph()
                    print(str(success) + ": " + str(instruction))
                    g = update_states(g, graph_state_list[i], object_id_map)
                    new_graph_state_list.append(copy.deepcopy(g))

                if os.path.exists("graph_state_list_with_bbox_png/" + out_dir_name + "_scene" + str(scene) + "/" + activity_name + "/" + str(activity_cnt)) == False:
                    os.mkdir("graph_state_list_with_bbox_png/" + out_dir_name + "_scene" + str(scene) + "/" + activity_name + "/" + str(activity_cnt))
                export(activity_name, new_graph_state_list, str(activity_cnt), time_list, new_script, object_name_map,  out_dir_name)
                
                activity_cnt += 1
            except Exception as e:
                print(e.args)
    except Exception as e:
        print(e.args)

results_text_rebuttal_specialparsed_programs_turk_third/split107_1.txt
['[Walk] <kitchen> (1)', '[Walk] <sink> (1)', '[Find] <faucet> (1)', '[SwitchOn] <faucet> (1)', '[Find] <dish_soap> (1)', '[Grab] <dish_soap> (1)', '[Pour] <dish_soap> (1) <sink> (1)', '[PutObjBack] <dish_soap> (1)', '[Find] <sponge> (1)', '[Grab] <sponge> (1)', '[Find] <plate> (1)', '[Grab] <plate> (1)', '[Scrub] <plate> (1)', '[Rinse] <plate> (1)', '[Find] <drying_rack> (1)', '[PutBack] <plate> (1) <drying_rack> (1)', '[Find] <plate> (2)', '[Grab] <plate> (2)', '[Scrub] <plate> (2)', '[Rinse] <plate> (2)', '[PutBack] <plate> (2) <drying_rack> (1)', '[PutObjBack] <sponge> (1)', '[SwitchOff] <faucet> (1)']
[{'is_off': ['faucet', '1']}, {'atreach': [['faucet', '1'], ['sink', '1']]}, {'atreach': [['dish_soap', '1'], ['sink', '1']]}, {'atreach': [['sponge', '1'], ['sink', '1']]}, {'atreach': [['plate', '1'], ['sink', '1']]}, {'atreach': [['drying_rack', '1'], ['sink', '1']]}, {'atreach': [['plate', '2'], ['sink', '1']]

results_text_rebuttal_specialparsed_programs_turk_third/split44_1.txt
['[Walk] <entrance_hall> (1)', '[Walk] <shoes> (1)', '[Find] <shoes> (1)', '[Find] <shoes> (1)', '[PutOff] <shoes> (1)', '[Walk] <closet> (1)', '[Open] <closet> (1)', '[Grab] <shoes> (1)', '[PutIn] <shoes> (1) <closet> (1)', '[Close] <closet> (1)']
results_text_rebuttal_specialparsed_programs_turk_third/split68_3.txt
['[Walk] <entrance_hall> (1)', '[Walk] <shoes> (1)', '[Find] <shoes> (1)', '[Find] <shoes> (1)', '[PutOff] <shoes> (1)', '[Grab] <shoes> (1)', '[Find] <floor> (1)', '[PutBack] <shoes> (1) <floor> (1)']
results_text_rebuttal_specialparsed_programs_turk_third/split70_1.txt
['[Walk] <entrance_hall> (1)', '[Walk] <shoes> (1)', '[Find] <shoes> (1)', '[Find] <shoes> (1)', '[PutOff] <shoes> (1)', '[Grab] <shoes> (1)', '[Find] <floor> (1)', '[PutBack] <shoes> (1) <floor> (1)']
results_text_rebuttal_specialparsed_programs_upwork_july/split11_10.txt
['[Walk] <kitchen> (1)', '[Walk] <fridge> (1)', '[Find] <fridge> 

Script is executable
name 'augmentation_utils' is not defined
object_id_map
{}
True: <char0> [WALK] <bedroom> (74)
True: <char0> [WALK] <dresser> (114)
True: <char0> [OPEN] <dresser> (114)
True: <char0> [FIND] <clothes_pants> (1001)
True: <char0> [GRAB] <clothes_pants> (1001)
True: <char0> [PULL] <clothes_pants> (1001)
True: <char0> [PUTIN] <clothes_pants> (1001) <dresser> (114)
True: <char0> [FIND] <clothes_pants> (1004)
True: <char0> [GRAB] <clothes_pants> (1004)
True: <char0> [PULL] <clothes_pants> (1004)
True: <char0> [PUTIN] <clothes_pants> (1004) <dresser> (114)
True: <char0> [FIND] <clothes_shirt> (1002)
True: <char0> [GRAB] <clothes_shirt> (1002)
True: <char0> [PUSH] <clothes_shirt> (1002)
True: <char0> [PUTIN] <clothes_shirt> (1002) <dresser> (114)
True: <char0> [FIND] <clothes_shirt> (1005)
True: <char0> [GRAB] <clothes_shirt> (1005)
True: <char0> [PUSH] <clothes_shirt> (1005)
True: <char0> [PUTIN] <clothes_shirt> (1005) <dresser> (114)
True: <char0> [FIND] <clothes_shirt> (1

Script is executable
name 'augmentation_utils' is not defined
object_id_map
{}
True: <char0> [WALK] <bedroom> (74)
True: <char0> [WALK] <mirror> (1002)
True: <char0> [FIND] <mirror> (1002)
True: <char0> [TURNTO] <mirror> (1002)
True: <char0> [LOOKAT] <mirror> (1002)
True: <char0> [FIND] <face> (1001)
True: <char0> [TURNTO] <face> (1001)
True: <char0> [LOOKAT] <face> (1001)
True: <char0> [FIND] <hair> (1003)
True: <char0> [TURNTO] <hair> (1003)
True: <char0> [LOOKAT] <hair> (1003)
True: <char0> [FIND] <clothes_dress> (1000)
True: <char0> [TURNTO] <clothes_dress> (1000)
True: <char0> [LOOKAT] <clothes_dress> (1000)
results_text_rebuttal_specialparsed_programs_upwork_july/split13_1.txt
['[Walk] <kitchen> (1)', '[Walk] <glass> (1)', '[Find] <glass> (1)', '[Grab] <glass> (1)', '[Walk] <fridge> (1)', '[Open] <fridge> (1)', '[Find] <milk> (1)', '[Grab] <milk> (1)', '[Pour] <milk> (1) <glass> (1)', '[PutObjBack] <milk> (1)', '[Close] <fridge> (1)']
[{'atreach': [['milk', '1'], ['fridge', '1']]

True: <char0> [FIND] <cup> (1001)
True: <char0> [GRAB] <cup> (1001)
True: <char0> [SCRUB] <cup> (1001)
True: <char0> [RINSE] <cup> (1001)
True: <char0> [PUTBACK] <cup> (1001) <dishrack> (1000)
True: <char0> [RINSE] <sponge> (1003)
True: <char0> [PUTOBJBACK] <sponge> (1003)
True: <char0> [SWITCHOFF] <faucet> (249)
results_text_rebuttal_specialparsed_programs_upworknturk_second/split81_2.txt
['[Walk] <bedroom> (1)', '[Walk] <closet> (1)', '[Find] <closet> (1)', '[Open] <closet> (1)', '[Find] <shoes> (1)', '[Grab] <shoes> (1)', '[PutOn] <shoes> (1)', '[Find] <clothes_jacket> (1)', '[Grab] <clothes_jacket> (1)', '[PutOn] <clothes_jacket> (1)', '[Close] <closet> (1)', '[Find] <cellphone> (1)', '[Grab] <cellphone> (1)']
results_text_rebuttal_specialparsed_programs_upwork_kellensecond/split9_8.txt
['[Walk] <entrance_hall> (1)', '[Walk] <shoes> (1)', '[Find] <shoes> (1)', '[Grab] <shoes> (1)', '[PutOn] <shoes> (1)', '[Find] <keys> (1)', '[Grab] <keys> (1)']
results_text_rebuttal_specialparsed_

True: <char0> [LOOKAT] <computer> (434)
True: <char0> [FIND] <check> (1000)
True: <char0> [GRAB] <check> (1000)
True: <char0> [READ] <check> (1000)
True: <char0> [FIND] <keyboard> (432)
True: <char0> [TYPE] <keyboard> (432)
True: <char0> [SWITCHOFF] <computer> (434)
results_text_rebuttal_specialparsed_programs_upworknturk_second/split10_3.txt
['[Walk] <entrance_hall> (1)', '[Walk] <centerpiece> (1)', '[Find] <centerpiece> (1)', '[Grab] <centerpiece> (1)', '[Walk] <table> (1)', '[PutBack] <centerpiece> (1) <table> (1)']
[{'location': [['centerpiece', '1'], ['entrance_hall', '1']]}, {'location': [['table', '1'], ['entrance_hall', '1']]}]
switch off
Script is executable
name 'augmentation_utils' is not defined
object_id_map
{}
True: <char0> [WALK] <home_office> (336)
True: <char0> [WALK] <centerpiece> (1000)
True: <char0> [FIND] <centerpiece> (1000)
True: <char0> [GRAB] <centerpiece> (1000)
True: <char0> [WALK] <table> (372)
True: <char0> [PUTBACK] <centerpiece> (1000) <table> (372)
resul

Script is executable
name 'augmentation_utils' is not defined
object_id_map
{}
True: <char0> [WALK] <bathroom> (11)
True: <char0> [WALK] <cleaning_solution> (1001)
True: <char0> [FIND] <cleaning_solution> (1001)
True: <char0> [GRAB] <cleaning_solution> (1001)
True: <char0> [WALK] <mirror> (1000)
True: <char0> [PUTOBJBACK] <cleaning_solution> (1001)
True: <char0> [FIND] <rag> (1002)
True: <char0> [GRAB] <rag> (1002)
True: <char0> [WALK] <mirror> (1000)
True: <char0> [WIPE] <mirror> (1000)
True: <char0> [WIPE] <mirror> (1000)
True: <char0> [WIPE] <mirror> (1000)
True: <char0> [PUTOBJBACK] <rag> (1002)
results_text_rebuttal_specialparsed_programs_upworknturk_second/split131_2.txt
['[Walk] <home_office> (1)', '[Walk] <chair> (1)', '[Find] <chair> (1)', '[Sit] <chair> (1)', '[Find] <computer> (1)', '[SwitchOn] <computer> (1)', '[TurnTo] <computer> (1)', '[LookAt] <computer> (1)', '[Find] <keyboard> (1)', '[Type] <keyboard> (1)']
[{'is_off': ['computer', '1']}, {'atreach': [['chair', '1'], [

True: <char0> [WALK] <home_office> (336)
True: <char0> [WALK] <desk> (374)
True: <char0> [FIND] <chair> (373)
True: <char0> [SIT] <chair> (373)
True: <char0> [FIND] <computer> (434)
True: <char0> [SWITCHON] <computer> (434)
True: <char0> [FIND] <keyboard> (432)
True: <char0> [TYPE] <keyboard> (432)
results_text_rebuttal_specialparsed_programs_upwork_kellensecond/split9_1.txt
['[Walk] <entrance_hall> (1)', '[Walk] <shoes> (1)', '[Find] <shoes> (1)', '[Find] <shoes> (1)', '[PutOff] <shoes> (1)', '[Walk] <kitchen> (1)', '[Walk] <groceries> (1)', '[Find] <groceries> (1)', '[Grab] <groceries> (1)', '[Find] <table> (1)', '[PutBack] <groceries> (1) <table> (1)']
results_intentions_march-13-18/file416_1.txt
['[Find] <sofa> (1)', '[Sit] <sofa> (1)', '[Lie] <sofa> (1)']
[{'free': ['sofa', '1']}]
switch off
('execute() takes 5 positional arguments but 6 were given',)
results_intentions_march-13-18/file204_2.txt
['[Walk] <living_room> (1)', '[Walk] <coffee> (1)', '[Find] <coffee> (1)', '[Grab] <co

results_intentions_march-13-18/file1027_2.txt
['[Walk] <bedroom> (1)', '[Walk] <bed> (1)', '[Lie] <bed> (1)', '[Sleep]']
results_intentions_march-13-18/file140_2.txt
['[Walk] <bedroom> (1)', '[Walk] <bed> (1)', '[Find] <bed> (1)', '[TurnTo] <bed> (1)', '[LookAt] <bed> (1)', '[Sit] <bed> (1)', '[Sleep]']
results_intentions_march-13-18/file244_2.txt
['[Walk] <bedroom> (1)', '[Walk] <bed> (1)', '[Lie] <bed> (1)', '[Sleep]']
results_intentions_march-13-18/file181_1.txt
['[Walk] <bedroom> (1)', '[Walk] <bed> (1)', '[Find] <bed> (1)', '[Lie] <bed> (1)', '[Sleep]']
results_intentions_march-13-18/file736_1.txt
['[Walk] <bedroom> (1)', '[Walk] <bed> (1)', '[Find] <bed> (1)', '[Lie] <bed> (1)', '[Find] <comforter> (1)', '[Pull] <comforter> (1)']
[{'atreach': [['bed', '1'], ['comforter', '1']]}, {'atreach': [['comforter', '1'], ['bed', '1']]}, {'location': [['bed', '1'], ['bedroom', '1']]}, {'location': [['comforter', '1'], ['bedroom', '1']]}, {'free': ['bed', '1']}]
switch off
Script is executab

KeyboardInterrupt: 

In [82]:
 len(executable_activity_list)

185

In [37]:
executable_activity_list[0]

{'activity_name': 'Go to sleep',
 'results': [{'file_name': 'results_intentions_march-13-18/file509_2.txt',
   'name': 'Go to sleep',
   'description': 'I walk into my bedroom. I change out of my clothing into pajamas. I turn off the light. I lie down in my bed and pull my sheets over me. I go to sleep.',
   'list_of_steps': ['[Walk] <bedroom> (1)',
    '[Walk] <clothes_pants> (1)',
    '[Find] <clothes_pants> (1)',
    '[Find] <clothes_pants> (1)',
    '[Find] <clothes_pants> (1)',
    '[PutOff] <clothes_pants> (1)',
    '[Find] <clothes_shirt> (1)',
    '[Find] <clothes_shirt> (1)',
    '[Find] <clothes_shirt> (1)',
    '[PutOff] <clothes_shirt> (1)',
    '[Find] <clothes_shirt> (2)',
    '[Grab] <clothes_shirt> (2)',
    '[PutOn] <clothes_shirt> (2)',
    '[Find] <clothes_pants> (2)',
    '[Grab] <clothes_pants> (2)',
    '[PutOn] <clothes_pants> (2)',
    '[Find] <light> (1)',
    '[SwitchOff] <light> (1)',
    '[Find] <bed> (1)',
    '[Lie] <bed> (1)',
    '[Find] <sheets> (1)',
 

In [40]:
cnt = 0
for executable_activity in rand_executable_activity_list:
    activity_list = executable_activity["results"]
    cnt += len(activity_list)
print(cnt)

680
